<a href="https://colab.research.google.com/github/clepigdo/Latihan_STKI2025/blob/main/DocumentSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Minggu 4/docSimilarity/tmdb_5000_movies.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [29]:
!pip install textsearch
!pip install contractions
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 546 to 4553
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        4800 non-null   object 
 1   tagline      4800 non-null   object 
 2   overview     4800 non-null   object 
 3   popularity   4800 non-null   float64
 4   description  4800 non-null   object 
dtypes: float64(1), object(4)
memory usage: 225.0+ KB


/tmp/ipython-input-1993753983.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.tagline.fillna('', inplace=True)


In [18]:
df.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


In [25]:
df.loc[546].description

'Before Gru, they had a history of bad bosses Minions Stuart, Kevin and Bob are recruited by Scarlet Overkill, a super-villain who, alongside her inventor husband Herb, hatches a plot to take over the world.'

# Bangun Sistem Rekomendasi Film

Tahapan
- Pre Processing
- Feature Engineering
- Komputasi Doc Similarity
- Proses Retrieve
- proses rekomendasi film


## Kemiripan Dokumen / document similarity

Ada berbagai cara untuk menghitung kesamaan antara dua item dokumen. Salah satu ukuran yang paling banyak digunakan adalah __cosine similarity__ .

### Cosine Similarity

Cosine Similarity digunakan untuk menghitung skor numerik untuk menunjukkan kesamaan antara dua dokumen teks. Secara matematis, ini didefinisikan sebagai berikut:

$$ cosinus(x,y) = \frac{x. y^\intercal}{||x||.||y||} $$

In [30]:
import nltk
import re
import numpy as np
import contractions


stop_words = nltk.corpus.stopwords.words('english') # Memuat daftar kata-kata umum Bahasa Inggris yang sering diabaikan dalam analisis (misalnya, 'the', 'is', 'and').

def normalize_document(doc):
    # Menghapus Karakter Khusus
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    # Lalu Mengubahnya ke Huruf Kecil
    doc = doc.lower()
    # Menghapus Spasi yang berlebihan
    doc = doc.strip()
    # Mengubah singkatan menjadi bentuk lengkap
    doc = contractions.fix(doc)
    # Memecah teks menjadi daftar kata individu
    tokens = nltk.word_tokenize(doc)
    # Menghapus kata(a,the,is) untuk mengurangi noise dan fokus pada kata kunci penting
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Mengubah kata kata yang sudah bersih tadi menjadi satu kalimat(string dokumen)
    doc = ' '.join(filtered_tokens)
    return doc
# Mengubah fungsi normalize_document menjadi fungsi yang vectorized
normalize_corpus = np.vectorize(normalize_document)

# Mengambil kolom description dari data frame
# Lalu menerapkan fungsi normalisasi ke setiap sinopsis film
# Dan hasilnya (Sinopsis yang sudah bersih) disimpan ke dalam variabel norm_corpus
norm_corpus = normalize_corpus(list(df['description']))

#Menghitung dan menampilkan jumlah total sinopsis film yang berhasil di proses
len(norm_corpus)

4800

In [32]:
print(df.shape)

(4800, 5)


Disini terlihat 4800 data berhasil di proses artinya semua data yang berada di dataset berhasil di proses

Selanjutnya Saya akan mengubah teks yang sudah bersih menjadi format numerik agar bisa dibaca oleh model, menggunakan TF-IDF (Term Frequency Inverse Document Frequency)

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer # Membuat Kosakata dari dokumen (Tokenisasi) dan menghitung bobot untuk setiap kata dalam dokumen

#ngram_range(1,2) disini menginstruksikan vectorizer untuk tidak hanya memakai kata tunggal tetapi juga pasangan kata
#Contoh : "film bagus keren" maka fitur yang akan dibuat adalah
# "film", "bagus", "keren" dan "film bagus", "bagus keren"
# ini dapat membantu menangkap konteks dan frasa yang lebih bermakna
tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2) # min_df=2 disini berperan untuk filter frekuensi dokumen (fitur/kata akan diabaikan jika muncul di kurang dari 2 dokumen )
# ini membantu mengurangi jumlah fitur dengan menghapus kata yang langka dan tdk signifikan
tfidf_matrix = tf.fit_transform(norm_corpus) # lalu mengubah semua teks sinopsis film yang sudah bersih menjadi matriks angka
tfidf_matrix.shape #Mengecek ukuran matriks

(4800, 20471)

Disini dapat dilihat:
4800 total sinopsis film dalam norm_corpus yang diproses
20471 total kata unik (unigram) dan pasangan kata unik (bigram) yang dijadikan kolom dalam matriks, setelah melewati filter min_df=2

Selanjutnya kita akan menghitung tingkat kemiripan antara setiap dokumen(Sinopsis Film) satu sama lain

In [34]:
from sklearn.metrics.pairwise import cosine_similarity # cosine_similarity digunakan untuk menghitung kemiripan antar vektor dalam space berdimensi tinggi yang dihasilkan oleh TF-IDF

doc_sim = cosine_similarity(tfidf_matrix) #Perhitungan Kemiripan dengan mengambil tfidf_matrix yang sudah diproses tadi
doc_sim_df = pd.DataFrame(doc_sim)
#doc_sim disini adalah sebuah matriks persegi dimana setiap nilai di dalamnya menunjukkan nilai kemiripan antara dua sinopsis film
# Nilai 1 berarti sinopsis kedua film sangat identik
# Nilai 0 berarti sinopsis kedua film sangat berbeda
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.000000,0.000000,0.000000,0.006071,0.008067,0.0,0.000000,0.0,0.000000,...,0.018758,0.000000,0.037930,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.000000,1.000000,0.000000,0.017839,0.007968,0.000000,0.0,0.012501,0.0,0.014840,...,0.000000,0.000000,0.017564,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.000000,0.000000,1.000000,0.000000,0.017178,0.000000,0.0,0.000000,0.0,0.024326,...,0.000000,0.006903,0.005024,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.000000,0.017839,0.000000,1.000000,0.000000,0.022414,0.0,0.000000,0.0,0.037207,...,0.000000,0.060846,0.025039,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.006071,0.007968,0.017178,0.000000,1.000000,0.004673,0.0,0.064581,0.0,0.000000,...,0.022064,0.019662,0.036561,0.0,0.015826,0.000000,0.076033,0.004516,0.043475,0.011465


In [35]:
# Mendapatkan title/judul film
movies_list = df['title'].values
movies_list, movies_list.shape

(array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
        'Alien Zone', 'America Is Still the Place'], dtype=object),
 (4800,))

Selanjutnya saya akan memilih salah satu film yaitu "Interstellar" untuk menemukan film yang serupa

In [42]:
movie_idx = np.where(movies_list == 'Interstellar')[0][0]
movie_idx

np.int64(1)

Lalu kita akan mengambil similariti yang cocok dengan film interstellar

In [43]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0.        , 1.        , 0.        , ..., 0.        , 0.        ,
       0.00796291])

Setelah itu saya akan mengambil 5 rekomendasi film yang mirip (Similar) dengan film Interstellar

In [44]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([ 219, 2215, 2105,  342,  816])

Lalu disini saya akan menampilkan judul apa saja yang ada di array yang muncul diatas

In [45]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['Gattaca', 'Space Pirate Captain Harlock', 'Space Cowboys',
       'Starship Troopers', 'Final Destination 2'], dtype=object)

Saya akan membuat rekomendasi 5 film, dari film apapun

In [46]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # Mencari id film
    movie_idx = np.where(movies == movie_title)[0][0]
    # Mencari Similar film
    movie_similarities = doc_sims.iloc[movie_idx].values
    # Mencari 5 film rekomendasi
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # Menampilkan nama dari index yang muncul
    similar_movies = movies[similar_movie_idxs]
    # Mengembalikan setiap nilai yang ada
    return similar_movies

Mari contohkan untuk mencari 5 rekomendasi film serupa

In [47]:
popular_movies = ['Minions', 'Interstellar', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys',
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice',
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [48]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Minions
Top 5 recommended Movies: ['Despicable Me 2' 'Despicable Me'
 'Teenage Mutant Ninja Turtles: Out of the Shadows' 'Superman'
 'Rise of the Guardians']

Movie: Interstellar
Top 5 recommended Movies: ['Gattaca' 'Space Pirate Captain Harlock' 'Space Cowboys'
 'Starship Troopers' 'Final Destination 2']

Movie: Deadpool
Top 5 recommended Movies: ['Silent Trigger' 'Underworld: Evolution' 'Bronson' 'Shaft' 'Don Jon']

Movie: Jurassic World
Top 5 recommended Movies: ['Jurassic Park' 'The Lost World: Jurassic Park'
 "National Lampoon's Vacation" 'The Nut Job' 'Vacation']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ["Pirates of the Caribbean: Dead Man's Chest"
 'Pirates of the Caribbean: On Stranger Tides' 'The Pirate'
 'The Pirates! In an Adventure with Scientists!' 'Joyful Noise']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Battle for the Planet of the Apes' 'Groove' 'The Other End of the Line'
 'Chicago Overcoat

KODE TAMBAHAN (MODEL LANJUTAN DARI TF-IDF) = FASTTEXT

In [50]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 66.6 MB/s eta 0:00:00


In [51]:
# Gensim FastText
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, window=30, min_count=2, workers=4, sg=1)

Fungsi ini mengambil representasi vektor dari setiap kata (yang dihasilkan oleh model FastText atau Word2Vec) dan merata-ratakannya untuk menghasilkan satu vektor tunggal yang mewakili makna keseluruhan satu dokumen.

In [52]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)

    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.

        for word in words:
            if word in vocabulary:
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

Angka 100 adalah nilai untuk parameter num_features (jumlah fitur). Ini menentukan dimensi atau panjang dari vektor numerik yang mewakili setiap kata dan setiap sinopsis film.

In [53]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 100)
doc_vecs_ft.shape

(4800, 100)

Mengecek Similarity

In [54]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.919827,0.969126,0.954164,0.966287,0.953822,0.985314,0.943288,0.973284,0.959846,...,0.972857,0.942426,0.980754,0.905749,0.911234,0.957443,0.978104,0.973325,0.956905,0.981285
1,0.919827,1.000000,0.965330,0.957324,0.961701,0.962074,0.940203,0.988819,0.951694,0.945999,...,0.886791,0.867707,0.934966,0.835951,0.837896,0.873573,0.887224,0.893891,0.880691,0.937696
2,0.969126,0.965330,1.000000,0.980635,0.984185,0.971201,0.976174,0.982717,0.985978,0.952889,...,0.944190,0.923879,0.956485,0.849146,0.900832,0.915779,0.962428,0.958267,0.939511,0.976293
3,0.954164,0.957324,0.980635,1.000000,0.980791,0.973490,0.975156,0.965486,0.970664,0.932421,...,0.950110,0.928012,0.947804,0.821668,0.905856,0.908876,0.958255,0.948827,0.935995,0.966062
4,0.966287,0.961701,0.984185,0.980791,1.000000,0.963683,0.977973,0.977341,0.976068,0.933863,...,0.958211,0.935788,0.958451,0.835412,0.919246,0.903552,0.965779,0.948023,0.957960,0.985714


In [55]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Minions
Top 5 recommended Movies: ['Quigley Down Under' 'The Fan' 'The Pursuit of D.B. Cooper'
 'Heroes of Dirt' 'The Mask']

Movie: Interstellar
Top 5 recommended Movies: ['The Inhabited Island' 'Sea Rex 3D: Journey to a Prehistoric World'
 'Gattaca' 'Beneath the Planet of the Apes'
 'Dawn of the Planet of the Apes']

Movie: Deadpool
Top 5 recommended Movies: ['Stake Land' 'Jakob the Liar' 'A Most Wanted Man'
 'Signed, Sealed, Delivered' 'The Girl with the Dragon Tattoo']

Movie: Jurassic World
Top 5 recommended Movies: ['Short Cut to Nirvana: Kumbh Mela' 'Samsara'
 'Dolphins and Whales: Tribes of the Ocean' '4 Months, 3 Weeks and 2 Days'
 'Jurassic Park']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ['Dylan Dog: Dead of Night' 'Tycoon' 'The Collection' 'Shanghai Surprise'
 'Extraordinary Measures']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Super Mario Bros.' 'Mutant World' 'Transformers: Dark of the Moon'
 'D